In [ ]:
# Mengimpor library
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import nltk
import sys


# Mendownload daftar kata yang ada (vocabulary)
nltk.download('stopwords')
from PyQt5 import QtWidgets, uic
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,recall_score,precision_score
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
indo = stopwords.words('indonesian')

# Mengimpor dataset
dataset = pd.read_csv('training_gojek.csv')


In [ ]:
# Melakukan proses cleaning pada teks
corpus = []
for i in range(0, len(dataset)):
    #re = hapus / rubah
    review = re.sub('[^a-zA-Z]', ' ', dataset['komentar'][i])
    review = review.lower()
    review = review.split()    
    # Menghilangkan kata yang tidak ada di stopwords
    psi = StemmerFactory()
    ps = psi.create_stemmer()
    review = [ps.stem(word) for word in review if not word in indo]
    print(i,review)
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
def hasil():
    #tf-idf
    articles=np.array(corpus)
    labels=np.array(dataset['sentimen'])

    tf_vectorizer=TfidfVectorizer(min_df=4,max_df=0.3,ngram_range=(1,3))
    x_train_tfidf=tf_vectorizer.fit_transform(articles).toarray()
    x_test_tfidf=tf_vectorizer.transform(articles)
    
    banyak_training=float(call.in_training.currentText())/100
    asd=float(banyak_training)
    X_train, X_test, y_train, y_test=train_test_split(x_train_tfidf,labels,test_size=banyak_training,random_state = 0)

    # Metode SVM
    in_kernel = call.in_kernel.currentText()
    classifierSVM = SVC(kernel = in_kernel, random_state = 0, gamma='auto')
    classifierSVM.fit(X_train, y_train)

    # Metode Naive Bayes
    classifierNB = GaussianNB()
    classifierNB.fit(X_train, y_train)

    # Memprediksi hasil Test Set
    y_pred_SVM = classifierSVM.predict(X_test)  # SVM
    y_pred_NB = classifierNB.predict(X_test)    # Naive Bayes

    # hasil
    svm_a = accuracy_score(y_test, y_pred_SVM, normalize=True)
    svm_r = recall_score(y_test, y_pred_SVM, average='micro')
    svm_p = precision_score(y_test, y_pred_SVM, average='micro')

    nbc_a = accuracy_score(y_test, y_pred_NB, normalize=True)
    nbc_r = recall_score(y_test, y_pred_NB, average='micro')
    nbc_p = precision_score(y_test, y_pred_NB, average='micro')

    call.svm_a.setText(str(svm_a))
    call.svm_r.setText(str(svm_r))
    call.svm_p.setText(str(svm_p))
    call.nbc_a.setText(str(nbc_a))
    call.nbc_r.setText(str(nbc_r))
    call.nbc_p.setText(str(nbc_p))

#     print("akurasi SVM = ", svm_a)
# print("akurasi NBC = ", nbc_a)

# print("recall SVM = ", svm_r)
# print("recall NBC = ", nbc_r)

# print("precision SVM = ", svm_p)
# print("precision NBC = ", nbc_p)

In [ ]:
app=QtWidgets.QApplication([])
call=uic.loadUi("gui.ui")

call.ok.clicked.connect(hasil)
call.show()
app.exec()